## Flight price scraping-HappyEasygo

##### I was not able to scrape the data as website will put the 'I am not a robot' check. This happened on all the website like cleartrip and skyscanner

In [1]:
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests

In [24]:
response=requests.get('https://www.happyeasygo.com')
response

<Response [200]>

In [3]:
source=['MAA','DEL','BOM']
destination=['BLR','COK','DEL','HYD']

date=['2021-11-04','2021-11-20','2021-12-15','2021-12-25','2022-01-01','2022-01-17','2022-02-03','2022-02-28','2022-03-14','2022-03-31','2022-04-20','2022-05-01',
      '2022-05-27','2022-06-06','2022-07-07']

In [ ]:
airline=[]
doj=[]
origin=[]
f_destination=[]
dep_time=[]
arr_time=[]
duration=[]
total_stop=[]
price=[]

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "eager" #since webpage has long loading time. This command will start scraping once 
driver = webdriver.Chrome(desired_capabilities=capa)  #the html is loaded.

for i in date:
    for j in source:
        for k in destination:
            if j!=k:
                url='https://www.happyeasygo.com/flights/{}-{}/{}?tripType=0&adults=1&childs=0&baby=0&cabinClass=Economy&airline=&carrier='.format(j,k,i)
                driver.get(url)
                time.sleep(5)
                
               #first we will scrape the data for non stop flights
                driver.find_element_by_xpath("//div[@class='lowest-price']").click()
                
                for l in driver.find_elements_by_xpath("//div[@class='flight-title P-ellipsis']"):
                    airline.append(l.text)
                    doj.append(i)
                    origin.append(j)
                    f_destination.append(k)
                    
                for l in driver.find_elements_by_xpath("//div[@class='o-vendor flight-stop ']"):
                    total_stop.append(l.text)
            
                for l in driver.find_elements_by_xpath("//span[@class='o-time-bold o-time_line']"):
                    dep_time.append(l.text)
                
                for l in driver.find_elements_by_xpath("//span[@class='o-time-bold']"):
                    arr_time.append(l.text.replace('\n+1',''))
                
                for l in driver.find_elements_by_xpath("//div[@class='o-vendor fly-time']"):
                    duration.append(l.text)
                    
                for l in driver.find_elements_by_xpath("//span[@class='price-from-second']"):
                    price.append(l.text.replace('₹ ','').replace(',',''))
                    
                    
                #now we will scrape the data from flights with one stop or more
                driver.find_element_by_xpath("//div[@class='lowest-price']").click()
                
                driver.find_element_by_xpath("//*[@id='stopsFilter']/li[2]/div[1]").click()
                driver.find_element_by_xpath("//*[@id='stopsFilter']/li[3]/div[1]").click()
                
                for l in driver.find_elements_by_xpath("//div[@class='flight-title P-ellipsis']"):
                    airline.append(l.text)
                    doj.append(i)
                    origin.append(j)
                    f_destination.append(k)
                    
            
                for l in driver.find_elements_by_xpath("//span[@class='o-time-bold o-time_line']"):
                    dep_time.append(l.text)
                
                for l in driver.find_elements_by_xpath("//span[@class='o-time-bold']"):
                    arr_time.append(l.text.replace('\n+1',''))
                
                for l in driver.find_elements_by_xpath("//div[@class='o-vendor fly-time']"):
                    duration.append(l.text)
                    
                for l in driver.find_elements_by_xpath("//span[@class='price-from-second']"):
                    price.append(l.text.replace('₹ ','').replace(',',''))       
                    
                for l in driver.find_elements_by_xpath("//div[@class='o-vendor flight-stop hasStop']"):
                    total_stop.append(l.text)
                
                    
                df=pd.DataFrame({'Airline':airline,'Date':doj,'Source':origin,'Destination':f_destination,'Dep_Time':dep_time,
                                 'Arrival_Time':arr_time,'Duration':duration,'Total_Stops':total_stop,'Price':price})
                df.to_csv('Flightdata.csv')
                print(len(price))

In [18]:
print(len(price))

153
